In [356]:
import pandas as pd
import numpy as np
from math import sin, cos, sqrt, atan2, radians


In [357]:
# path to CSV file
csv_file_path = 'JC-202307-citibike-tripdata.csv'

# Load the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_file_path)

df.head()  # Display the first few rows of the DataFrame

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7A68381621C25F78,classic_bike,2023-07-17 17:16:34,2023-07-17 17:20:52,Astor Place,JC077,Communipaw & Berry Lane,JC084,40.719225,-74.071281,40.714358,-74.066611,member
1,0F814CA67B2FA120,classic_bike,2023-07-26 19:40:15,2023-07-26 19:44:37,Adams St & 2 St,HB407,9 St HBLR - Jackson St & 8 St,HB305,40.739727,-74.036866,40.747907,-74.038412,member
2,775A38967EBF5FB4,electric_bike,2023-07-01 12:12:22,2023-07-01 12:27:45,McGinley Square,JC055,Riverview Park,JC057,40.725340,-74.067622,40.744319,-74.043991,member
3,D93B742DCE1C1447,classic_bike,2023-07-20 19:10:18,2023-07-20 19:17:22,Baldwin at Montgomery,JC020,Brunswick St,JC023,40.723455,-74.064359,40.724176,-74.050656,member
4,AA7A6863B4B92169,electric_bike,2023-07-07 19:33:59,2023-07-07 19:58:17,Baldwin at Montgomery,JC020,Mama Johnson Field - 4 St & Jackson St,HB404,40.723659,-74.064194,40.743140,-74.040041,casual


In [358]:
record_count = df.shape[0]
print("Record count:", record_count)


Record count: 106608


In [359]:

# List of CSV file names
csv_files = ['JC-202208-citibike-tripdata.csv', 'JC-202209-citibike-tripdata.csv', 'JC-202210-citibike-tripdata.csv', 'JC-202211-citibike-tripdata.csv', 'JC-202212-citibike-tripdata.csv',
             'JC-202301-citibike-tripdata.csv', 'JC-202302-citibike-tripdata.csv', 'JC-202303-citibike-tripdata.csv', 'JC-202304-citibike-tripdata.csv', 'JC-202305-citibike-tripdata.csv',
             'JC-202306-citibike-tripdata.csv','JC-202307-citibike-tripdata.csv',
             'JC-202207-citibike-tripdata.csv','JC-202206-citibike-tripdata.csv',
             'JC-202205-citibike-tripdata.csv','JC-202204-citibike-tripdata.csv','JC-202203-citibike-tripdata.csv','JC-202202-citibike-tripdata.csv',
             'JC-202201-citibike-tripdata.csv',
             'JC-202112-citibike-tripdata.csv','JC-202111-citibike-tripdata.csv','JC-202110-citibike-tripdata.csv',
             'JC-202109-citibike-tripdata.csv','JC-202108-citibike-tripdata.csv'
             ]

# Initialize an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Loop through each CSV file and concatenate it to the combined DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Now, combined_df contains all the data from the ten CSV files


In [360]:
record_count = combined_df.shape[0]
print("Record count:", record_count)

Record count: 1834144


In [361]:
# Remove duplicates based on all columns
df_no_duplicates = combined_df.drop_duplicates()

In [362]:
record_count = df_no_duplicates.shape[0]
print("Record count:", record_count)

Record count: 1834144


In [363]:
# Delete rows where 'rideable_type' equals "docked_bike"
filtered_combined_df = df_no_duplicates[df_no_duplicates['rideable_type'] != 'docked_bike']


In [364]:
record_count = filtered_combined_df.shape[0]
print("Record count:", record_count)

Record count: 1819962


In [365]:
# Count the total number of records (rows) with null values
total_null_records = filtered_combined_df.isnull().any(axis=1).sum()

print("Total records with null values:", total_null_records)

Total records with null values: 7075


In [366]:

filtered_combined_df = filtered_combined_df.dropna()

In [367]:
# Count the total number of records (rows) with null values
total_null_records = filtered_combined_df.isnull().any(axis=1).sum()

print("Total records with null values:", total_null_records)

Total records with null values: 0


In [368]:
record_count = filtered_combined_df.shape[0]
print("Record count:", record_count)

Record count: 1812887


In [369]:
# Define the haversine_distance function
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    # Differences in latitude and longitude
    Δlat = lat2 - lat1
    Δlon = lon2 - lon1

    # Haversine formula
    a = sin(Δlat / 2)**2 + cos(lat1) * cos(lat2) * sin(Δlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Radius of the Earth (mean radius in miles)
    R = 3959.0

    # Calculate the distance in miles
    distance = R * c

    return distance

# Calculate distances in miles and add them to the DataFrame
distances = []
for index, row in filtered_combined_df.iterrows():
    start_lat = row['start_lat']
    start_lon = row['start_lng']
    end_lat = row['end_lat']
    end_lon = row['end_lng']
    
    distance = haversine_distance(start_lat, start_lon, end_lat, end_lon)
    distances.append(distance)

filtered_combined_df['distance_miles'] = distances




In [370]:
# Now, df contains the distances in miles in the 'distance_miles' column
filtered_combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_miles
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.718355,-74.038914,40.727596,-74.044247,member,0.696916
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.736982,-74.027781,40.745984,-74.028199,member,0.622379
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.736982,-74.027781,40.749985,-74.027150,member,0.899060
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.718355,-74.038914,40.719420,-74.050990,member,0.636667
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.736982,-74.027781,40.742677,-74.051789,member,1.317073


In [371]:
record_count = filtered_combined_df.shape[0]
print("Record count:", record_count)

Record count: 1812887


In [372]:
# Get data types of columns
column_data_types = filtered_combined_df.dtypes

print(column_data_types)

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
distance_miles        float64
dtype: object


In [373]:
# Convert the date-time columns to datetime objects
filtered_combined_df['started_at'] = pd.to_datetime(filtered_combined_df['started_at'])
filtered_combined_df['ended_at'] = pd.to_datetime(filtered_combined_df['ended_at'])



In [374]:
# Get data types of columns
column_data_types = filtered_combined_df.dtypes

print(column_data_types)

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
distance_miles               float64
dtype: object


In [375]:
filtered_combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_miles
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.718355,-74.038914,40.727596,-74.044247,member,0.696916
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.736982,-74.027781,40.745984,-74.028199,member,0.622379
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.736982,-74.027781,40.749985,-74.027150,member,0.899060
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.718355,-74.038914,40.719420,-74.050990,member,0.636667
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.736982,-74.027781,40.742677,-74.051789,member,1.317073


In [376]:
# Calculate the time difference in minutes
filtered_combined_df['time_with_bike_minutes'] = (filtered_combined_df['ended_at'] - filtered_combined_df['started_at']).dt.total_seconds() / 60


In [377]:
filtered_combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_miles,time_with_bike_minutes
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.718355,-74.038914,40.727596,-74.044247,member,0.696916,6.633333
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.736982,-74.027781,40.745984,-74.028199,member,0.622379,9.516667
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.736982,-74.027781,40.749985,-74.027150,member,0.899060,79.033333
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.718355,-74.038914,40.719420,-74.050990,member,0.636667,10.133333
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.736982,-74.027781,40.742677,-74.051789,member,1.317073,19.316667


In [378]:
# Export the combined DataFrame to a new CSV file
filtered_combined_df.to_csv('combined_data.csv', index=False)

Route data starts here

In [379]:
route_df = filtered_combined_df

route_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_miles,time_with_bike_minutes
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.718355,-74.038914,40.727596,-74.044247,member,0.696916,6.633333
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.736982,-74.027781,40.745984,-74.028199,member,0.622379,9.516667
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.736982,-74.027781,40.749985,-74.027150,member,0.899060,79.033333
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.718355,-74.038914,40.719420,-74.050990,member,0.636667,10.133333
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.736982,-74.027781,40.742677,-74.051789,member,1.317073,19.316667


In [380]:
# Get data types of columns
column_data_types = route_df.dtypes

print(column_data_types)

ride_id                           object
rideable_type                     object
started_at                datetime64[ns]
ended_at                  datetime64[ns]
start_station_name                object
start_station_id                  object
end_station_name                  object
end_station_id                    object
start_lat                        float64
start_lng                        float64
end_lat                          float64
end_lng                          float64
member_casual                     object
distance_miles                   float64
time_with_bike_minutes           float64
dtype: object


In [381]:

route_df['start_lat'] = route_df['start_lat'].astype(str)
route_df['start_lng'] = route_df['start_lng'].astype(str)
route_df['end_lat'] = route_df['end_lat'].astype(str)
route_df['end_lng'] = route_df['end_lng'].astype(str)



In [382]:
# Get data types of columns
column_data_types = route_df.dtypes

print(column_data_types)

ride_id                           object
rideable_type                     object
started_at                datetime64[ns]
ended_at                  datetime64[ns]
start_station_name                object
start_station_id                  object
end_station_name                  object
end_station_id                    object
start_lat                         object
start_lng                         object
end_lat                           object
end_lng                           object
member_casual                     object
distance_miles                   float64
time_with_bike_minutes           float64
dtype: object


In [383]:

route_df['route'] = route_df['rideable_type'] + '_' + route_df['start_lat'] + '_' + route_df['start_lng'] \
    + '_' + route_df['end_lat'] + '_' + route_df['end_lng'] + '_' + route_df['member_casual']

In [384]:
route_df.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_miles,time_with_bike_minutes,route
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.71835519823214,-74.03891444206238,40.727595966,-74.044247311,member,0.696916,6.633333,classic_bike_40.71835519823214_-74.03891444206...
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.73698221818716,-74.02778059244156,40.745983882915624,-74.02819901704788,member,0.622379,9.516667,classic_bike_40.73698221818716_-74.02778059244...
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.73698221818716,-74.02778059244156,40.74998490907138,-74.02715027332306,member,0.899060,79.033333,classic_bike_40.73698221818716_-74.02778059244...
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.71835519823214,-74.03891444206238,40.71942,-74.05099,member,0.636667,10.133333,classic_bike_40.71835519823214_-74.03891444206...
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.73698221818716,-74.02778059244156,40.742677141,-74.051788633,member,1.317073,19.316667,electric_bike_40.73698221818716_-74.0277805924...


In [385]:
# Count the total number of records (rows) with null values
total_null_records = route_df.isnull().any(axis=1).sum()

print("Total records with null values:", total_null_records)

Total records with null values: 0


In [386]:
route_df['route_count'] = route_df.groupby('route')['route'].transform('count')


In [387]:
route_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,distance_miles,time_with_bike_minutes,route,route_count
0,D68047AAA9A72501,classic_bike,2022-08-12 18:14:45,2022-08-12 18:21:23,Columbus Drive,JC014,Hamilton Park,JC009,40.71835519823214,-74.03891444206238,40.727595966,-74.044247311,member,0.696916,6.633333,classic_bike_40.71835519823214_-74.03891444206...,381
1,E5B82A9F588A7650,classic_bike,2022-08-19 01:04:39,2022-08-19 01:14:10,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,8 St & Washington St,HB603,40.73698221818716,-74.02778059244156,40.745983882915624,-74.02819901704788,member,0.622379,9.516667,classic_bike_40.73698221818716_-74.02778059244...,961
2,13D19CCBCC9D7B76,classic_bike,2022-08-19 18:13:45,2022-08-19 19:32:47,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,11 St & Washington St,HB502,40.73698221818716,-74.02778059244156,40.74998490907138,-74.02715027332306,member,0.899060,79.033333,classic_bike_40.73698221818716_-74.02778059244...,1498
3,AA0DBFF220DA98F6,classic_bike,2022-08-21 13:43:05,2022-08-21 13:53:13,Columbus Drive,JC014,Montgomery St,JC099,40.71835519823214,-74.03891444206238,40.71942,-74.05099,member,0.636667,10.133333,classic_bike_40.71835519823214_-74.03891444206...,273
4,8F11D340492FCD32,electric_bike,2022-08-04 22:02:12,2022-08-04 22:21:31,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Pershing Field,JC024,40.73698221818716,-74.02778059244156,40.742677141,-74.051788633,member,1.317073,19.316667,electric_bike_40.73698221818716_-74.0277805924...,48


In [388]:
# Calculate the threshold for what percentage of records to get
threshold = route_df['route_count'].quantile(0.5)

print(threshold)

180.0


In [389]:
# Filter the DataFrame to include only rows with Value_Count greater than or equal to the threshold
route_df_filtered = route_df[route_df['route_count'] >= threshold]


In [390]:
# Select specific columns to create a new DataFrame

selected_columns = ['rideable_type','start_lat','start_lng','end_lat','end_lng','member_casual','route','route_count']


new_df = route_df_filtered[selected_columns]



In [391]:
new_df.head()

,rideable_type,start_lat,start_lng,end_lat,end_lng,member_casual,route,route_count
0,classic_bike,40.71835519823214,-74.03891444206238,40.727595966,-74.044247311,member,classic_bike_40.71835519823214_-74.03891444206...,381
1,classic_bike,40.73698221818716,-74.02778059244156,40.745983882915624,-74.02819901704788,member,classic_bike_40.73698221818716_-74.02778059244...,961
2,classic_bike,40.73698221818716,-74.02778059244156,40.74998490907138,-74.02715027332306,member,classic_bike_40.73698221818716_-74.02778059244...,1498
3,classic_bike,40.71835519823214,-74.03891444206238,40.71942,-74.05099,member,classic_bike_40.71835519823214_-74.03891444206...,273
8,classic_bike,40.71835519823214,-74.03891444206238,40.7241765,-74.0506564,member,classic_bike_40.71835519823214_-74.03891444206...,253


In [392]:
record_count = new_df.shape[0]
print("Record count:", record_count)

Record count: 906773


In [393]:
# Remove duplicates based on all columns
new_df = new_df.drop_duplicates()

In [394]:
record_count = new_df.shape[0]
print("Record count:", record_count)

Record count: 2003


In [395]:

new_df.to_csv('top_routes.csv', index=False)
